<a href="https://colab.research.google.com/github/RFOXgithub/Sistem-Rekomendasi-Pengelolaan-Produk/blob/main/Final_Project_Deployment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.1 MB/s eta 0:00:00


In [3]:
%%writefile app.py
import streamlit as st
import pickle
import pandas as pd
import numpy as np

# Load model
with open('rf_modelStock.pkl', 'rb') as f:
    rf_modelStock = pickle.load(f)

with open('rf_modelPrice.pkl', 'rb') as f:
    rf_modelPrice = pickle.load(f)

with open('rf_modelPopu.pkl', 'rb') as f:
    rf_modelPopu = pickle.load(f)

# Kategori produk
categories = [
    'Elektronik', 'Lainnya', 'Perawatan Pribadi', 'Olahraga & Outdoor',
    'Peralatan Rumah Tangga', 'Pakaian & Fashion', 'Kendaraan & Aksesori',
    'Makanan & Minuman', 'Perhiasan & Aksesori', 'Mainan & Anak-Anak',
    'Gadget & Elektronik Musik'
]

# Fungsi prediksi
def prediction(df):
    df['avg_harga_per_kategori'] = df.groupby('kategori')['harga'].transform('mean')
    df['harga_per_rating'] = df['harga'] / (df['total_rating'] + 1)
    df['harga_terjual'] = df['harga'] * df['terjual']
    df['rasio_penjualan_stok'] = df['terjual'] / (df['stock'] + 1)
    df['stok_terjual_ratio'] = df['stock'] / (df['terjual'] + 1)
    df['stok_ideal'] = df['stok_terjual_ratio'] * df['stock']
    df['stok_ideal'] = np.ceil(df['stok_ideal'])

    df['harga_kategori_encoding'] = rf_modelPrice.predict(
        df[['avg_harga_per_kategori', 'harga_per_rating', 'harga_terjual']]
    )
    df['restock_encoding'] = rf_modelStock.predict(
        df[['stok_ideal', 'stok_terjual_ratio', 'rasio_penjualan_stok']]
    )
    df['popularitas_encoding'] = rf_modelPopu.predict(
        df[['harga_per_rating', 'rasio_penjualan_stok', 'total_rating']]
    )

    harga_kategori_mapping = {0: 'Rendah', 1: 'Sedang', 2: 'Tinggi'}
    restock_mapping = {1: 'Tidak Restock', 2: 'Restock', 0: 'Stok Berlebih'}
    popularitas_mapping = {0: 'Tidak Populer', 1: 'Populer', 2: 'Sangat Populer'}

    df['harga_kategori'] = df['harga_kategori_encoding'].map(harga_kategori_mapping)
    df['restock'] = df['restock_encoding'].map(restock_mapping)
    df['popularitas'] = df['popularitas_encoding'].map(popularitas_mapping)

    def rekomendasi(row):
        if row['restock'] == 'Restock':
            return "Segera lakukan restock produk ini."
        elif row['harga_kategori'] == 'Tinggi' and row['popularitas'] == 'Sangat Populer':
            return "Lakukan promosi pada produk populer ini."
        elif row['restock'] == 'Tidak Restock' and row['harga_kategori'] == 'Rendah':
            return "Evaluasi produk untuk diskon atau hapus dari katalog."
        elif row['popularitas'] == 'Tidak Populer' and row['restock'] == 'Stok Berlebih':
            return "Tunda restock produk ini dan evaluasi penjualannya."
        elif row['popularitas'] == 'Populer' and row['harga_kategori'] == 'Sedang':
            return "Pertahankan produk dengan harga dan popularitas saat ini."
        else:
            return "Pertahankan strategi saat ini."

    df['Rekomendasi'] = df.apply(rekomendasi, axis=1)
    return df[['nama_produk', 'harga_kategori', 'restock', 'popularitas', 'Rekomendasi']]

# Antarmuka Streamlit
def main():
    st.title("Sistem Rekomendasi Produk")
    st.markdown("""
    <div style="background-color:yellow;padding:13px">
    <h1 style="color:black; text-align:center;">Aplikasi Analisis dan Rekomendasi Produk</h1>
    </div>
    """, unsafe_allow_html=True)

    with st.form(key='my_form'):
        nama_produk = st.text_input("Nama Produk")
        kategori = st.selectbox("Pilih Kategori", categories)
        harga = st.number_input("Harga", min_value=0, step=1000)
        total_rating = st.number_input("Total Rating", min_value=0, step=1)
        min_terjual = total_rating if total_rating > 0 else 0
        terjual = st.number_input("Jumlah Terjual", min_value=min_terjual, step=1)
        stock = st.number_input("Stock", min_value=0, step=1)

        # Tombol submit form
        submit_button = st.form_submit_button(label="Proses Rekomendasi")

    # Logika setelah tombol diklik
    if submit_button:
        kategori_index = categories.index(kategori) + 1  # Konversi ke indeks kategori numerik
        data_input = {
            'nama_produk': [nama_produk],
            'kategori': [kategori_index],
            'harga': [harga],
            'total_rating': [total_rating],
            'terjual': [terjual],
            'stock': [stock]
        }
        df = pd.DataFrame(data_input)
        hasil = prediction(df)
        st.success("Berikut adalah hasil analisis produk:")
        st.dataframe(hasil)
    else:
        st.info("Isi form di atas dan tekan tombol 'Proses Rekomendasi' untuk melihat hasil analisis.")

if __name__ == '__main__':
    main()


Writing app.py


In [4]:
!wget -q -O - ipv4.icanhazip.com

34.106.67.148


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501



⠙⠹⠸
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.106.67.148:8501

⠼⠴⠦Need to install the following packages:
localtunnel@2.0.2
Ok to proceed? (y) y

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙your url is: https://early-bananas-share.loca.lt
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.4.2 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator RandomForestClassifier from version 1.4.2 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own ri